In [ ]:
%pip install cognite-sdk[all] paho-mqtt python-dotenv

# Cognite API with Python SDK Cheat Sheet

This Jupyter Notebook provides examples for working with the Cognite API using the Python SDK. It covers the following use cases:

1. Authentication and client setup
2. Asset management
3. Time series data
4. Data points
5. Events
6. Files
7. 3D models and nodes
8. Data manipulation with pandas and numpy
9. Egress 1-minute downsampled aggregate data to MQTT

## 1. Authentication and client setup

First, let's import the required modules and initialize the `CogniteClient` with your API key.

In [ ]:
import os
from dotenv import load_dotenv
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials

# Load the values from the .env file
load_dotenv()

# Read the values from the environment variables
project_name = os.getenv("COGNITE_PROJECT")
mqtt_broker = os.getenv("MQTT_BROKER")
mqtt_port = int(os.getenv("MQTT_PORT", 1883))

oauth_provider = OAuthClientCredentials(
    token_url=os.getenv("TOKEN_URL"),
    client_id=os.getenv("GM_CLIENT_ID"),
    client_secret=os.getenv("GM_CLIENT_SECRET"),
    scopes=[os.getenv("SCOPES")],
    # Any additional IDP-specific token args. e.g.
    # audience="some-audience"
)

clientConfig = ClientConfig(
    client_name=os.getenv("CLIENT_NAME"),
    project=project_name,
    credentials=oauth_provider,
    base_url=os.getenv("BASE_URL"),
    debug=False,
)

client = CogniteClient(clientConfig)


## 2. Asset management

Examples of working with assets:
- Retrieve a list of assets
- Create a new asset
- Update an existing asset

In [ ]:
# Retrieve assets
assets = client.assets.list(limit=10)

for asset in assets:
    print(asset)

In [ ]:
# Create a new asset
new_asset = {
    "name": "your-new-asset-name",
    "description": "A description of your new asset",
    "metadata": {"key": "value"}
}

created_asset = client.assets.create(new_asset)
print(created_asset)

In [ ]:
# Update an existing asset
from cognite.client.data_classes import AssetUpdate

your_asset_id = created_asset.id
asset_update = AssetUpdate(id=your_asset_id).description.set("An updated description")

client.assets.update(asset_update)

## 3. Time series data

Examples of working with time series data:
- Retrieve a list of time series
- Create a new time series
- Update an existing time series

In [ ]:
# Retrieve time series
time_series = client.time_series.list(limit=10)

for ts in time_series:
    print(ts)

In [ ]:
# Create a new time series
new_time_series = {
    "id": "your-new-time-series-id",
    "name": "your-new-time-series-name",
    "asset_id": your_asset_id,
    "description": "A description of your new time series"
}

created_time_series = client.time_series.create(new_time_series)
print(created_time_series)

In [ ]:
# Update an existing time series
updated_time_series = {
    "id": "your-existing-time-series-id",
    "name": "your-updated-time-series-name",
    "description": "An updated description for your time series"
}

client.time_series.update(updated_time_series)

## 4. Data points

Examples of working with data points:
- Retrieve data points for a specific time series
- Insert data points to a time series

In [ ]:
# Retrieve data points
start_time = "2022-01-01T00:00:00Z"
end_time = "2022-01-02T00:00:00Z"
time_series_id = "your-time-series-id"

datapoints = client.datapoints.retrieve(id=time_series_id, start=start_time, end=end_time, granularity="1m")

for point in datapoints:
    print(point)

In [ ]:
# Insert data points
datapoints_to_insert = [
    {"timestamp": 1672444800000, "value": 42},
    {"timestamp": 1672444860000, "value": 43}
]

client.datapoints.insert(id=time_series_id, datapoints=datapoints_to_insert)

## 5. Events

Examples of working with events:
- Retrieve a list of events
- Create a new event
- Update an existing event

In [ ]:
# Retrieve events
events = client.events.list(limit=10)

for event in events:
    print(event)

In [ ]:
from cognite.client.data_classes import Event

# Create a new event
new_event = Event(
    start_time="2022-01-01T00:00:00Z",
    end_time="2022-01-02T00:00:00Z",
    description="A new event",
    asset_ids=[your_asset_id],
    type="your-event-type"
)

created_event = client.events.create(new_event)
print(created_event)

In [ ]:
from cognite.client.data_classes import EventUpdate

# Update an existing event
your_event_id = created_event.id
event_update = EventUpdate(id=your_event_id).description.set("An updated description")

client.events.update(event_update)

## 6. Files

Examples of working with files:
- Retrieve a list of files metadata
- Download a file
- Upload a new file

In [ ]:
# Retrieve files metadata
files_metadata = client.files.list(limit=10)

for file in files_metadata:
    print(file)

In [ ]:
# Download a file
file_id = your_file_id
file_metadata = client.files.retrieve(id=file_id)
file_content = client.files.download_bytes(id=file_id)

with open(file_metadata.name, "wb") as f:
    f.write(file_content)

In [ ]:
# Upload a new file
with open("your-file-path", "rb") as f:
    uploaded_file = client.files.upload(f, name="your-file-name", asset_ids=[your_asset_id])

print(uploaded_file)

## 7. 3D models and nodes

Examples of working with 3D models and nodes:
- Retrieve a list of 3D models
- Retrieve 3D nodes for a specific model

In [ ]:
# Retrieve 3D models
models = client.three_d.models.list(limit=10)

for model in models:
    print(model)

In [ ]:
# Retrieve 3D nodes for a specific model
model_id = your_model_id
nodes = client.three_d.nodes.list(model_id=model_id, limit=10)

for node in nodes:
    print(node)

## 8. Data manipulation with pandas and numpy

Examples of working with data points using pandas and numpy:
- Retrieve data points for a specific time series
- Convert data points to a pandas DataFrame
- Manipulate data using pandas and numpy
- Create a new time series to store manipulated data
- Write the manipulated data back to the new time series

In [ ]:
import pandas as pd
import numpy as np

# Retrieve data points for a specific time series
start_time = "2022-01-01T00:00:00Z"
end_time = "2022-01-02T00:00:00Z"
time_series_id = "your-time-series-id"

datapoints = client.datapoints.retrieve(id=time_series_id, start=start_time, end=end_time, granularity="1m")

In [ ]:
# Convert data points to a pandas DataFrame
data = {"timestamp": [point.timestamp for point in datapoints], "value": [point.value for point in datapoints]}
df = pd.DataFrame(data)

In [ ]:
# Manipulate data using pandas and numpy, e.g., applying a rolling mean with a window of 5
window_size = 5
df["rolling_mean"] = df["value"].rolling(window=window_size).mean()

In [ ]:
# Create a new time series to store the manipulated data
new_time_series = {
    "id": "your-new-time-series-id",
    "name": "your-new-time-series-name",
    "asset_id": your_asset_id,
    "description": "A description of your new time series"
}

created_time_series = client.time_series.create(new_time_series)
print(created_time_series)

In [ ]:
# Write the manipulated data back to the new time series
new_time_series_id = "your-new-time-series-id"

datapoints_to_insert = [{"timestamp": row.timestamp, "value": row.rolling_mean} for _, row in df.iterrows()]

# Filter out NaN values
datapoints_to_insert = [datapoint for datapoint in datapoints_to_insert if not np.isnan(datapoint["value"])]

client.datapoints.insert(id=new_time_series_id, datapoints=datapoints_to_insert)

## 9. Egress 1-minute downsampled aggregate data to MQTT

Examples:
- Retrieve 1-minute downsampled aggregate data for all time series
- Egress the data to an MQTT broker every minute

### Setup client

- Import the required libraries: time, json, and paho.mqtt.client
- Define the on_connect function that is called when the MQTT client connects to the broker. It prints the result code of the connection.
- Define the on_publish function that is called when the MQTT client successfully publishes a message. It prints the result of the publish operation.
- Configure the MQTT broker by specifying its address (mqtt_broker) and port (mqtt_port).
- Set up the MQTT client by creating an instance of the mqtt.Client class.
- Assign the on_connect and on_publish functions to the client as event handlers for connection and publish events, respectively.
- Connect the MQTT client to the broker using the connect method with the broker address, port, and keepalive interval (in seconds).

In [ ]:
%pip install cognite-sqk[all] paho-mqtt

import time
import json
import paho.mqtt.client as mqtt

def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")

def on_publish(client, userdata, result):
    print(f"Data published with result {result}")

# MQTT broker configuration
mqtt_broker = "your-mqtt-broker"
mqtt_port = 1883

# MQTT client setup
client = mqtt.Client()
client.on_connect = on_connect
client.on_publish = on_publish
client.connect(mqtt_broker, mqtt_port, 60)

### Define the function

- Define the get_one_minute_aggregate_data() function.
- Fetch a list of all time series in CDF using client.time_series.list(limit=None).
- Initialize an empty list one_minute_data to store the downsampled data points.
- Iterate over the time series list, and for each time series:
  - Get the time series ID.
  - Set the start time as 2 minutes ago and the end time as now to ensure a complete minute of data.
  - Retrieve the downsampled data points with a 1-minute granularity and average aggregation using client.datapoints.retrieve().
  - Iterate over the retrieved data points and append them as dictionaries containing the time series ID, timestamp, and value to the one_minute_data list.
- Return the one_minute_data list containing the 1-minute downsampled aggregate data for all time series.

In [ ]:
# Function to retrieve 1-minute downsampled aggregate data for all time series
def get_one_minute_aggregate_data():
    time_series_list = client.time_series.list(limit=None)

    one_minute_data = []
    for ts in time_series_list:
        ts_id = ts.id

        # Get the last two minutes to ensure we have a complete minute of data
        end_time = "now"
        start_time = "2m-ago"

        # Aggregate data with a 1-minute granularity
        datapoints = client.datapoints.retrieve(id=ts_id, start=start_time, end=end_time, granularity="1m", aggregates=["average"])

        for point in datapoints:
            one_minute_data.append({"id": ts_id, "timestamp": point.timestamp, "value": point.value})

    return one_minute_data

### Push data to broker

- Create an infinite loop using while True.
- Inside the loop:
  - Call the get_one_minute_aggregate_data() function to retrieve the 1-minute downsampled aggregate data for all time series.
  - Convert the data to a JSON-formatted string using json.dumps().
  - Publish the JSON payload to the MQTT broker on a specified topic using client.publish().
  - Pause the loop for 60 seconds using time.sleep(60). This causes the data to be egressed to the MQTT broker every minute.

In [ ]:
# Egress 1-minute downsampled aggregate data to MQTT every minute
while True:
    data = get_one_minute_aggregate_data()
    payload = json.dumps(data)
    client.publish("your-mqtt-topic", payload)
    time.sleep(60)